# Singapore Public Housing (HDB) Resale Price Prediction Model (Part 1)
### Data Collection - Housing Address Coordinates

## 1. Initialization

In [1]:
# Import Vanilla Libraries
import requests, json, time, random, math
import pandas as pd
import numpy as np

In [2]:
# Read property dataset as dataframe
hdb = pd.read_csv('./Dataset/Raw/hdb_2017_2020.csv')

In [3]:
hdb.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


## 2. Feature Engineering

### 2.1 Date Time Data

In [4]:
# Extract Month and Year from 'month' column
hdb['sold_year'] = hdb['month'].str[:4].astype(int)
hdb['sold_month'] = hdb['month'].str[-2:].astype(int)

In [5]:
# Function to convert 'remaining lease' to numerical column
def extract_lease_years(row):
    if len(row)==4:
        return round(int(row[0]) + int(row[2])/12, 2)
    else:
        return int(row[0])

In [6]:
# Apply function to 'remaining lease'
hdb['remaining_lease'] = hdb['remaining_lease'].str.split().apply(extract_lease_years)

In [7]:
hdb.sample(5)

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,sold_year,sold_month
9065,2017-06,PUNGGOL,5 ROOM,289C,PUNGGOL PL,13 TO 15,112.0,Premium Apartment,2013,94.75,600000.0,2017,6
7692,2017-05,TOA PAYOH,5 ROOM,10,JOO SENG RD,16 TO 18,119.0,Improved,1983,64.92,572888.0,2017,5
8622,2017-06,HOUGANG,4 ROOM,514,HOUGANG AVE 10,01 TO 03,89.0,Simplified,1986,67.67,390000.0,2017,6
27893,2018-05,TAMPINES,5 ROOM,126,TAMPINES ST 11,10 TO 12,140.0,Model A,1985,66.42,580000.0,2018,5
31325,2018-07,JURONG WEST,5 ROOM,351,CORPORATION DR,07 TO 09,121.0,Improved,1997,78.50,450000.0,2018,7


In [8]:
# Concatenate block number and street name to form full address for API Calls
hdb['address'] = hdb['block'] + ' ' + hdb['street_name']

In [9]:
# Drop all the pre-engineered column
hdb.drop(['month', 'block', 'street_name'], axis=1, inplace=True)

### 2.2 Storey Range

In [10]:
# Check on storey range
hdb.storey_range.unique()

array(['10 TO 12', '01 TO 03', '04 TO 06', '07 TO 09', '13 TO 15',
       '19 TO 21', '22 TO 24', '16 TO 18', '34 TO 36', '28 TO 30',
       '37 TO 39', '49 TO 51', '25 TO 27', '40 TO 42', '31 TO 33',
       '46 TO 48', '43 TO 45'], dtype=object)

In [11]:
# Convert 'storey_range' into numerical by taking the midpoint of each range
hdb.storey_range = (hdb.storey_range.str[:2].astype(int) + hdb.storey_range.str[-2:].astype(int))
hdb.storey_range = (hdb.storey_range/2).astype(int)

In [12]:
# Sanity check
hdb.head()

,town,flat_type,storey_range,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,sold_year,sold_month,address
0,ANG MO KIO,2 ROOM,11,44.0,Improved,1979,61.33,232000.0,2017,1,406 ANG MO KIO AVE 10
1,ANG MO KIO,3 ROOM,2,67.0,New Generation,1978,60.58,250000.0,2017,1,108 ANG MO KIO AVE 4
2,ANG MO KIO,3 ROOM,2,67.0,New Generation,1980,62.42,262000.0,2017,1,602 ANG MO KIO AVE 5
3,ANG MO KIO,3 ROOM,5,68.0,New Generation,1980,62.08,265000.0,2017,1,465 ANG MO KIO AVE 10
4,ANG MO KIO,3 ROOM,2,67.0,New Generation,1980,62.42,265000.0,2017,1,601 ANG MO KIO AVE 5


### 2.3 Coordinates

#### 2.3.1 OneMap API Calling

In [13]:
unique_add = pd.Series(hdb['address'].unique())

In [14]:
unique_add = pd.DataFrame(unique_add, columns=["address"])

In [15]:
# Extract coordinates from OneMap API by sending full address of each block
latitude = []
longitude = []

if 'latitude' not in unique_add.columns:
    unique_add['latitude'] = np.nan
    unique_add['longitude'] = np.nan

for i, address in enumerate(unique_add['address']):
    if math.isnan(unique_add.loc[i, 'latitude']):
        try:
            print('\rWaiting... ({})... {} addresses remaining... '.format(address, len(unique_add)-i-1), end='.')
            query = "https://developers.onemap.sg/commonapi/search?searchVal=" + address + "&returnGeom=Y&getAddrDetails=N"
            response = requests.get(query)
            coor_json = json.loads(response.content)
            unique_add.loc[i, 'latitude'] = coor_json['results'][0]['LATITUDE']
            unique_add.loc[i, 'longitude'] = coor_json['results'][0]['LONGITUDE']

        except:
            unique_add.loc[i, 'latitude'] = np.nan
            unique_add.loc[i, 'longitude'] = np.nan

        # Export dataframe every loop to csv for inspection
        unique_add.to_csv('./Dataset/Transitional/unique_address.csv', index=False)

        # Sleeping time to avoid overloading server
        time.sleep(random.randint(1,2)/4)

if unique_add['latitude'].isnull().sum() == 0:
    print("--- Data is complete ---")

Waiting... (666A YISHUN AVE 4)... 0 addresses remaining... . . ... ...

#### 2.3.2 Addressing Missing Data

In [16]:
unique_add[unique_add['latitude'].isnull()]['address'].unique()

array(["3 ST. GEORGE'S RD", "21 ST. GEORGE'S RD", "11 ST. GEORGE'S RD",
       "8 ST. GEORGE'S LANE", "18 ST. GEORGE'S RD", "15 ST. GEORGE'S RD",
       "9 ST. GEORGE'S RD", "4B ST. GEORGE'S LANE", "7 ST. GEORGE'S LANE",
       "5 ST. GEORGE'S LANE", "22 ST. GEORGE'S RD", "20 ST. GEORGE'S RD",
       "13 ST. GEORGE'S RD", "6 ST. GEORGE'S LANE", "2 ST. GEORGE'S RD",
       "14 ST. GEORGE'S RD", "23 ST. GEORGE'S RD", "16 ST. GEORGE'S RD",
       "1 ST. GEORGE'S RD", "10 ST. GEORGE'S RD", "17 ST. GEORGE'S RD"],
      dtype=object)

In [17]:
missing_index = unique_add[unique_add['latitude'].isnull()].index

In [18]:
unique_add.loc[missing_index, 'address'] = unique_add.loc[missing_index, 'address'].str.replace('ST.', 'SAINT')

In [19]:
unique_add['latitude'] = unique_add['latitude'].astype(float)
unique_add['longitude'] = unique_add['longitude'].astype(float)

#### 2.3.3 Second Round API Calling

In [20]:
# Repassing dataframe for API Calling but only missing data is processed
for i, lat in enumerate(unique_add.latitude):
    
    # Only if latitude is missing, fire up API calling query
    if math.isnan(lat):
        address = unique_add.loc[i, 'address']
        try:
            print('\rWaiting... ({})... {} neighbourhood remaining... '.format(address, len(unique_add)-i-1), end='.')
            query = "https://developers.onemap.sg/commonapi/search?searchVal=" + address + "&returnGeom=Y&getAddrDetails=N"
            response = requests.get(query)
            coor_json = json.loads(response.content)
            unique_add.loc[i, 'latitude'] = coor_json['results'][0]['LATITUDE']
            unique_add.loc[i, 'longitude'] = coor_json['results'][0]['LONGITUDE']
            time.sleep(random.randint(1,2)/4)

        except:
            unique_add.loc[i, 'latitude'] = np.nan
            unique_add.loc[i, 'longitude'] = np.nan
            time.sleep(random.randint(1,2)/4)
            
unique_add.to_csv('./Dataset/Transitional/unique_address.csv', index=False)

if unique_add['latitude'].isnull().sum() == 0:
    print("--- Data is complete ---")

Waiting... (17 SAINT GEORGE'S RD)... 616 neighbourhood remaining... ....--- Data is complete ---


In [21]:
# Sanity check on missing data
unique_add = pd.read_csv("./Dataset/Transitional/unique_address.csv")

In [22]:
unique_add.isnull().sum()

address      0
latitude     0
longitude    0
dtype: int64

## 3. Final Data Export

In [24]:
hdb = pd.merge(hdb, unique_add, how='left', on='address')

In [26]:
# Export final dataframe to CSV
hdb.to_csv('./Dataset/Transitional/complete_data.csv', index=False)

In [27]:
# Sanity check on the final dataframe
hdb = pd.read_csv('./Dataset/Transitional/complete_data.csv')
hdb.isnull().sum()

town                     0
flat_type                0
storey_range             0
floor_area_sqm           0
flat_model               0
lease_commence_date      0
remaining_lease          0
resale_price             0
sold_year                0
sold_month               0
address                  0
latitude               164
longitude              164
dtype: int64

In [31]:
missing_data = hdb[hdb['latitude'].isnull()].index
hdb.loc[missing_data, 'address'] = hdb.loc[missing_data, 'address'].str.replace('ST.', 'SAINT')
hdb = pd.merge(hdb, unique_add, how='left', on='address')

In [32]:
hdb.isnull().sum()

town                     0
flat_type                0
storey_range             0
floor_area_sqm           0
flat_model               0
lease_commence_date      0
remaining_lease          0
resale_price             0
sold_year                0
sold_month               0
address                  0
latitude_x             164
longitude_x            164
latitude_y               0
longitude_y              0
dtype: int64

In [33]:
hdb.drop(['latitude_x', 'longitude_x'], axis=1, inplace=True)
hdb.rename(columns={'latitude_y': 'latitude', 'longitude_y': 'longitude'}, inplace=True)

In [37]:
# Export final dataframe to CSV
hdb.to_csv('./Dataset/Transitional/complete_data.csv', index=False)